In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

Aim of this Kernel is to visually understand what is happening around the world concerning COVID-19. To do this, it is being used data coming from the dataset itself and data coming from John Hopkings Univerity (available  at the following link: https://github.com/CSSEGISandData/COVID-19. 

I have also added other information and dataset as the population density, population middle age per country and the country iso codes to be able to compare data with other indicators.

Before starting with the visualizations, please, let me know all your feedbacks, thus, I could improve the work and the learnship through this Kernel.


In [ ]:
confirmed_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
deaths_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
country_codes = pd.read_csv("../input/countries-iso-codes/wikipedia-iso-country-codes.csv")
population_density = pd.read_csv("../input/population-density/population_density.csv", delimiter=";",error_bad_lines = False, index_col = "Country Code")
population_middle_age = pd.read_csv("../input/population-middle-age/Population_middle_age.csv", delimiter=",",error_bad_lines = False, index_col = "Place")

In [ ]:
population_middle_age = population_middle_age.Median

In [ ]:
Pop_dens = "2018"
population_density = population_density.loc[:,Pop_dens]

In [ ]:
confirmed_df = pd.merge(confirmed_df, country_codes, right_on = "English short name lower case", left_on = "Country/Region")
confirmed_df = pd.merge(confirmed_df, population_density, right_index = True, left_on = "Alpha-3 code")
confirmed_df = pd.merge(confirmed_df, population_middle_age, right_index = True, left_on = "Country/Region")
deaths_df = pd.merge(deaths_df, country_codes, right_on = "English short name lower case", left_on = "Country/Region")
deaths_df = pd.merge(deaths_df, population_density, right_index = True, left_on = "Alpha-3 code")

In [ ]:
cols = confirmed_df.keys()
Last_Date = cols[-8]
codes = cols[-5]
First_Date = cols[4]

In [ ]:
confirmed_df = confirmed_df[confirmed_df[Last_Date]>=400]
deaths_df = deaths_df[deaths_df[Last_Date]>=4]

In [ ]:
deaths_df = deaths_df.groupby("Country/Region").apply(lambda df: df.iloc[:,4:].sum()).sort_values(by=Last_Date)
confirmed_df = confirmed_df.groupby("Country/Region").apply(lambda df: df.iloc[:,4:].sum()).sort_values(by=Last_Date)

In [ ]:
#df.replace({'A': {0: 100, 4: 400}})
#confirmed_df = confirmed_df.replace({"Alpha-3 code": {"China": "CHN", "Canada":"CAN", "Australia":"AUS"}})
#confirmed_df.replace({confirmed_df.at["China","Alpha-3 code"]:"CHN"})
#confirmed_df.tail(30)
#confirmed_df.at["China","Alpha-3 code"]
confirmed_df.loc[confirmed_df['Alpha-3 code'] ==confirmed_df.at["China","Alpha-3 code"], 'Alpha-3 code'] = 'CHN'
confirmed_df.loc[confirmed_df['Alpha-3 code'] ==confirmed_df.at["Canada","Alpha-3 code"], 'Alpha-3 code'] = 'CAN'
confirmed_df.loc[confirmed_df['Alpha-3 code'] ==confirmed_df.at["Australia","Alpha-3 code"], 'Alpha-3 code'] = 'AUS'
deaths_df.loc[deaths_df['Alpha-3 code'] ==deaths_df.at["Australia","Alpha-3 code"], 'Alpha-3 code'] = 'AUS'
deaths_df.loc[deaths_df['Alpha-3 code'] ==deaths_df.at["China","Alpha-3 code"], 'Alpha-3 code'] = 'CHN'
deaths_df.loc[deaths_df['Alpha-3 code'] ==deaths_df.at["Canada","Alpha-3 code"], 'Alpha-3 code'] = 'CAN'
rate_dead_df = pd.Series(deaths_df.loc[:,Last_Date]/confirmed_df.loc[:,Last_Date]*100).dropna().sort_values()

First exercise to comprehend data we have is to verify which are these countries with most infected people by the disease and to show up in the following plots. 

The relation between dead people and confirmed people is important to see the behaviour of the disease across different countries. Plot reflecting this information is the third one.

In [ ]:
import plotly.express as px
fig = px.bar(confirmed_df, x = confirmed_df.index , y = Last_Date, color = Last_Date )
fig.show()
fig_dead = px.bar(deaths_df, x = deaths_df.index , y = Last_Date, color = Last_Date )
fig_dead.show()
fig_rate = px.bar(rate_dead_df, x = rate_dead_df.index, y = Last_Date, color = Last_Date)
fig_rate.show()

In [ ]:
df_analisys = pd.DataFrame([confirmed_df["Alpha-3 code"],
                            confirmed_df[Last_Date],
                            deaths_df[Last_Date],
                            rate_dead_df,
                            confirmed_df.Median,
                            confirmed_df[Pop_dens]]).transpose().fillna(1)
df_analisys.columns = ["Country Codes" , "Confirmed", "Dead", "Rate Dead", "Median" ,"Population Density"]

To have an easier understanding, following is the same information but instead of a comparation, in a map to see the geographical distribution.

In [ ]:
fig_map = px.choropleth(df_analisys, locations = "Country Codes",
                       color = "Confirmed",
                       hover_name = df_analisys.index,
                       color_continuous_scale=px.colors.sequential.Plasma)
fig_dead_map = px.choropleth(df_analisys, locations = "Country Codes",
                       color = "Dead",
                       hover_name = df_analisys.index,
                       color_continuous_scale=px.colors.sequential.Plasma)
fig_ratedead_map = px.choropleth(df_analisys, locations = "Country Codes",
                       color = "Rate Dead",
                       hover_name = df_analisys.index,
                       color_continuous_scale=px.colors.sequential.Plasma)
fig_map.show()
fig_dead_map.show()
fig_ratedead_map.show()

What I can see is that the Paretto's rules is accomplished (20% of the countrues represent the 80% of the cases). Among the first 6 (7 contries if we consider US which is not appearing in data) countries are represented most of the cases around the globe. This countries are:
 - United States (Is not appearing)
 - Italy
 - China
 - Spain
 - Germany
 - France
 - Iran
 
What we can observe is 6/7 countries are inside of the highest economies in the world. So, my questions are:
 - Relation between confirmed cases and dead cases. More confirmed cases means more dead rate? 
 - How is related the density of poblation of the countries respect to the disease spreading
 - Relation between population middle age and confirmed, dead rate indicator
 
Concerning to relate the confirmed and dead rate indicator against middle age is coming cause from what I can understand, this disease is affecting much more older people than younger, so, could it be this countries with higher economier which are also presenting older people, has more mortality because of that?

In [ ]:
fig_dense = px.scatter(df_analisys, x="Confirmed", y="Population Density",
                 size='Dead',hover_name = df_analisys.index, trendline ="ols")
fig_b = px.scatter(df_analisys, x = "Rate Dead", y= "Confirmed", size = "Dead", color = "Population Density",hover_name = df_analisys.index, trendline ="ols")
fig_dense.show()
fig_b.show()

Let's analyze how are related the average age per country against affected and dead people by COVID-19 (also with the dead rate). 

In [ ]:
df_analisys.loc[df_analisys['Median'] ==df_analisys.at["China","Median"], 'Median'] = 37.4
fig = px.scatter(df_analisys, x = "Median", y= "Rate Dead", size = "Dead", color = "Population Density",trendline ="ols", hover_name = df_analisys.index)
fig.show()
fig_b = px.scatter(df_analisys, x = "Median", y= "Confirmed", size = "Dead", color = "Population Density",trendline ="ols", hover_name = df_analisys.index)
fig_b.show()

We can observe that the relation is not clear enough in the comparations done. R coefficient from the linear regressions done is very low in every graph shown, so, we cannot say it exists a correlation.

In [ ]:
Countries_ = ["Italy", "China", "Spain", "Iran", "Germany", "France"]
df_top_infec = confirmed_df.loc[Countries_,First_Date : Last_Date]
df_top_dead = deaths_df.loc[Countries_,First_Date : Last_Date]

It could be possible to see the trends of the different countries (more affected):

 - Confirmed trend cases
 - Dead people trend cases

In [ ]:
import plotly.graph_objects as go

fig_count = go.Figure()
# Create and style traces
fig_count.add_trace(go.Scatter(x=df_top_infec.columns, y=df_top_infec.loc[Countries_[0]], name=Countries_[0],
                            line=dict(color='Green', width=4)))
fig_count.add_trace(go.Scatter(x=df_top_infec.columns, y=df_top_infec.loc[Countries_[1]], name=Countries_[1],
                            line=dict(color='Yellow', width=4)))
fig_count.add_trace(go.Scatter(x=df_top_infec.columns, y=df_top_infec.loc[Countries_[2]], name=Countries_[2],
                            line=dict(color='Red', width=4)))
fig_count.add_trace(go.Scatter(x=df_top_infec.columns, y=df_top_infec.loc[Countries_[3]], name=Countries_[3],
                            line=dict(color='Brown', width=4)))
fig_count.add_trace(go.Scatter(x=df_top_infec.columns, y=df_top_infec.loc[Countries_[4]], name=Countries_[4],
                            line=dict(color='Black', width=4)))
fig_count.add_trace(go.Scatter(x=df_top_infec.columns, y=df_top_infec.loc[Countries_[5]], name=Countries_[5],
                            line=dict(color='Blue', width=4)))
fig_count.show()

In [ ]:
fig_count = go.Figure()
# Create and style traces
fig_count.add_trace(go.Scatter(x=df_top_dead.columns, y=df_top_dead.loc[Countries_[0]], name=Countries_[0],
                            line=dict(color='Green', width=4)))
fig_count.add_trace(go.Scatter(x=df_top_dead.columns, y=df_top_dead.loc[Countries_[1]], name=Countries_[1],
                            line=dict(color='Yellow', width=4)))
fig_count.add_trace(go.Scatter(x=df_top_dead.columns, y=df_top_dead.loc[Countries_[2]], name=Countries_[2],
                            line=dict(color='Red', width=4)))
fig_count.add_trace(go.Scatter(x=df_top_dead.columns, y=df_top_dead.loc[Countries_[3]], name=Countries_[3],
                            line=dict(color='Brown', width=4)))
fig_count.add_trace(go.Scatter(x=df_top_dead.columns, y=df_top_dead.loc[Countries_[4]], name=Countries_[4],
                            line=dict(color='Black', width=4)))
fig_count.add_trace(go.Scatter(x=df_top_dead.columns, y=df_top_dead.loc[Countries_[5]], name=Countries_[5],
                            line=dict(color='Blue', width=4)))
fig_count.show()

What we can observe is whereas China was able to control the pandemic, european countries are presenting difficulties to control it despite strong actions they have taken it. It is also important to undestand that affected and confirmed cases are not evolving more in these countries if not it seems that now are lines, not exponential curves as they were when the pandemic started. So, it is time to see the quantity of people affected to decrease.

In [ ]:
df_data = pd.read_csv("../input/novel-corona-virus-2019-dataset/COVID19_line_list_data.csv", index_col = "id",)
df_data_open = pd.read_csv("../input/novel-corona-virus-2019-dataset/COVID19_open_line_list.csv", parse_dates = ["date_admission_hospital","date_death_or_discharge"])

Now, let's analyze the dataset provided to see which information we can explore:

Ok, In this Dataset I have following information:
 - Reporting Date (It seems when the disease started)
 - Country (Also, location, it could help us for followings analysis, let's keep it)
 - Gender
 - Age
 
Well, Ok, with this information we could responde following questions:
 - Age rate for getting and infecting COVID-19. Objective of this information is to know how many population of each age rate (per decade) is affected per coronavirus as we know that older people is sustancially more affected than younger people.
 - Gender affected. To see how this disease affects people regarding their gender.

In [ ]:
def correct_sex(x):
    if x == "Male" or x == "male":
        return "Male"
    elif x == "female" or x == "Female":
        return "Female"
    else:
        return np.nan
df_gender = df_data_open.sex.apply(correct_sex).value_counts()
import plotly.express as px
fig = px.bar(df_gender, x = df_gender.index ,y = "sex", color = "sex")
fig.show()

In [ ]:
def correct_age(row):
    try:
        y = int(row)
        return y
    except:
        return np.nan
df_age = df_data_open.age.apply(correct_age)

In [ ]:
def correct_outcome(x):
    if x == "discharged" or x == "discharge" or x == "Discharged":
        return "Discharged"
    elif x == "died" or x == "death":
        return "Death"
    elif x == "recovered" or x =="stable":
        return "Recovered"
    else:
        return np.nan
df_outcome = df_data_open.outcome.apply(correct_outcome)

In [ ]:
df_ = pd.DataFrame([df_age,
                    df_outcome ,
                    df_data_open.date_admission_hospital, 
                    df_data_open.date_death_or_discharge,
                    df_data_open.country],).transpose()

In [ ]:
df_.dropna(subset = ["age"])
df_country_ =df_.groupby("country").country.count()

In [ ]:
df_outcome_ = df_.groupby("outcome").outcome.count()
fig = px.pie(df_outcome_, names = df_outcome_.index,values = "outcome", title = "Situation of people infected by COVID-19")
fig.show()

Lets try to do different clusters of age. Lets divide it in clustes of 10 years starting at 0 and finishing at 100. Thus, 10 clusters. So, in this way the clusters are the followings:

 - 1: People between 0 and 10 years old
 - 2: People between 10 and 20 years old
 - 3: People between 20 and 30 years old
 - 4: People between 30 and 40 years old 
 - 5: People between 40 and 50 years old
 - 6: People between 50 and 60 years old
 - 7: People between 60 and 70 years old
 - 8: People between 70 and 80 years old
 - 9: People between 80 and 90 years old
 - 10: People between 90 and 100 years old
 


In [ ]:
df_dn_outcome = df_.dropna(subset = ["date_death_or_discharge"])

In [ ]:
def group_age(x):
    if x >= 90:
        return 10
    elif x >=80:
        return 9
    elif x >=70:
        return 8
    elif x >=60:
        return 7
    elif x >=50:
        return 6
    elif x >=40:
        return 5
    elif x >=30:
        return 4
    elif x >=20:
        return 3
    elif x >=10:
        return 2
    elif x >=0:
        return 1
df_age = df_.dropna(subset = ["age"])
df_age_ = df_age.age.apply(group_age).value_counts()
fig = px.bar(df_age_, x = df_age_.index,y = "age")
fig.show()